In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Spark book")\
.getOrCreate()

22/11/18 22:51:53 WARN Utils: Your hostname, qtt-HP-EliteBook-840-G6 resolves to a loopback address: 127.0.1.1; using 192.168.1.32 instead (on interface wlp58s0)
22/11/18 22:51:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/18 22:51:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
static = spark.read.json("./data/activity-data/")
dataSchema = static.schema

In [3]:
dataSchema

StructType([StructField('Arrival_Time', LongType(), True), StructField('Creation_Time', LongType(), True), StructField('Device', StringType(), True), StructField('Index', LongType(), True), StructField('Model', StringType(), True), StructField('User', StringType(), True), StructField('gt', StringType(), True), StructField('x', DoubleType(), True), StructField('y', DoubleType(), True), StructField('z', DoubleType(), True)])

In [4]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.json("./data/activity-data")

In [5]:
activityCounts = streaming.groupBy("gt").count()

In [6]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [7]:
activityQuery = activityCounts.writeStream.queryName("activity_counts")\
.format("memory").outputMode("complete")\
.start()

22/11/18 22:52:21 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f9a98758-6b04-4e23-99ff-520cdfb483a9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/18 22:52:21 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
from time import sleep
for x in range(5):
    spark.sql("SELECT * FROM activity_counts").show()
    sleep(1)

+----------+------+
|        gt| count|
+----------+------+
|       sit|910858|
|     stand|842480|
|stairsdown|692880|
|      walk|980872|
|  stairsup|773870|
|      null|773032|
|      bike|798933|
+----------+------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 935476|
|     stand| 865247|
|stairsdown| 711606|
|      walk|1007382|
|  stairsup| 794780|
|      null| 793930|
|      bike| 820526|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 960096|
|     stand| 888015|
|stairsdown| 730332|
|      walk|1033892|
|  stairsup| 815690|
|      null| 814826|
|      bike| 842118|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|

In [9]:
from pyspark.sql.functions import expr
simpleTransform = streaming.withColumn("stairs", expr("gt like '%stairs%'"))\
.where("stairs")\
.where("gt is not null")\
.select("gt", "model", "arrival_time", "creation_time")\
.writeStream\
.queryName("simple_transform")\
.format("memory")\
.outputMode("append")\
.start()

22/11/18 23:04:24 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-343ae3d9-b6d8-4462-805a-4751df600eb1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/18 23:04:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [10]:
deviceModelStats = streaming.cube("gt", "model").avg()\
.drop("avg(Arrival_time)")\
.drop("avg(Creation_Time)")\
.drop("avg(Index)")\
.writeStream.queryName("device_counts").format("memory")\
.outputMode("complete")\
.start()

22/11/18 23:08:29 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-03346ba6-5336-400a-908c-b03619a52fe7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/18 23:08:29 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [12]:
spark.sql("SELECT * FROM device_counts").show()

+----------+------+--------------------+--------------------+--------------------+
|        gt| model|              avg(x)|              avg(y)|              avg(z)|
+----------+------+--------------------+--------------------+--------------------+
|       sit|  null|-5.49433244039590...|2.791446281700068E-4|-2.33994461689890...|
|      walk|nexus4|-0.00390116006094368|0.001052508689953...|-6.95435553042992...|
|      walk|  null|-0.00390116006094368|0.001052508689953...|-6.95435553042992...|
|  stairsup|  null|-0.02479965287771635|-0.00800392344379...|  -0.100340884150604|
|     stand|  null|-3.11082189691724...|3.218461665975318E-4|2.141300040636475...|
|      bike|  null|0.022688759550866824|-0.00877912156368...|-0.08251001663412387|
|  stairsup|nexus4|-0.02479965287771635|-0.00800392344379...|  -0.100340884150604|
|      null|nexus4|4.796918779024537E-4|-0.00601540958963...|-0.01013356489164804|
|      null|  null|4.796918779024537E-4|-0.00601540958963...|-0.01013356489164804|
|sta

In [15]:
historicalAgg = static.groupBy("gt", "model").avg()
deviceModelStats = streaming.drop("Arrival_Time", "Creation_Time", "Index")\
.cube("gt", "model").avg()\
.join(historicalAgg, ["gt", "model"])\
.writeStream.queryName("device_count").format("memory")\
.outputMode("complete")\
.start()

22/11/18 23:11:37 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-baa36dfa-66b4-4646-97b1-a3b4db72cd0f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/18 23:11:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [16]:
df1 = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "host1:port1,host2:port2")\
.option("subscribe", "topic1")\
.load()

AnalysisException:  Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".        

In [19]:
spark.sql("SELECT * FROM device_count").show()

+----------+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+
|        gt| model|              avg(x)|              avg(y)|              avg(z)|   avg(Arrival_Time)|  avg(Creation_Time)|        avg(Index)|              avg(x)|              avg(y)|              avg(z)|
+----------+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+
|      bike|nexus4| 0.02286620854109702|-0.00976932276685...|-0.08249503213481174|1.424751134339986...|1.424752127369588...| 326459.6867328154| 0.02268875955086685|-0.00877912156368...|-0.08251001663412341|
|      null|nexus4|-0.00763194754145...|-1.27732722151030...|0.002400666320012182|1.424749002876339...|1.424749919482128...| 219276.9663669269|-0.00847688860109...|-7.30455